In [6]:
import cv2
import numpy as np
import tensorflow as tf

In [7]:
detections_global = []
selected_box = None
tracking = False
tracker = None

In [8]:
interpreter = tf.lite.Interpreter(model_path="detect.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

height = input_details[0]['shape'][1]
width = input_details[0]['shape'][2]

In [9]:
labels = []
with open("labelmap.txt","r") as f:
    for line in f:
        labels.append(line.strip())

In [1]:
import cv2
import numpy as np
import tensorflow as tf

detections_global = []
selected_box = None
tracking = False
tracker = None
current_frame = None

interpreter = tf.lite.Interpreter(model_path="detect.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

height = input_details[0]['shape'][1]
width  = input_details[0]['shape'][2]

labels = []
with open("labelmap.txt", "r") as f:
    for line in f:
        labels.append(line.strip())

def compute_iou(boxA, boxB):
    ax1, ay1, ax2, ay2 = boxA
    bx1, by1, bx2, by2 = boxB

    inter_x1 = max(ax1, bx1)
    inter_y1 = max(ay1, by1)
    inter_x2 = min(ax2, bx2)
    inter_y2 = min(ay2, by2)

    inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)
    area_a = (ax2 - ax1) * (ay2 - ay1)
    area_b = (bx2 - bx1) * (by2 - by1)
    union = area_a + area_b - inter_area

    return inter_area / union if union > 0 else 0


def mouse_click(event, x, y, flags, param):
    global selected_box, tracking, tracker

    if event == cv2.EVENT_LBUTTONDOWN:
        for box in detections_global:
            x1, y1, x2, y2, label = box
            if x1 <= x <= x2 and y1 <= y <= y2:
                selected_box = (x1, y1, x2, y2)
                tracking = True
                tracker = cv2.legacy.TrackerKCF_create()
                tracker.init(current_frame, (x1, y1, x2 - x1, y2 - y1))
                print(f"Now tracking: {label}")
                break


cap = cv2.VideoCapture(0)
cv2.namedWindow("Object Detection")
cv2.setMouseCallback("Object Detection", mouse_click)

while True:
    ret, frame = cap.read()
    if not ret:                      
        break

    current_frame = frame.copy()
    detections_global = []           

    h, w, _ = frame.shape

    img = cv2.resize(frame, (width, height))
    input_data = np.expand_dims(img, axis=0).astype(np.uint8)

    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()

    boxes   = interpreter.get_tensor(output_details[0]['index'])[0]
    classes = interpreter.get_tensor(output_details[1]['index'])[0]
    scores  = interpreter.get_tensor(output_details[2]['index'])[0]

    for i in range(len(scores)):
        if scores[i] > 0.5:
            ymin, xmin, ymax, xmax = boxes[i]

            x1 = int(xmin * w)
            y1 = int(ymin * h)
            x2 = int(xmax * w)
            y2 = int(ymax * h)

            label = labels[int(classes[i])]
            detections_global.append((x1, y1, x2, y2, label))

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            if label != '???':
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    if tracking and tracker is not None:
        success, bbox = tracker.update(frame)

        if success:
            tx, ty, tw, th = [int(v) for v in bbox]
            selected_box = (tx, ty, tx + tw, ty + th)

            cv2.rectangle(frame, (tx, ty), (tx + tw, ty + th), (0, 245, 160), 3)
            cv2.putText(frame, "TRACKING", (tx, ty - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 245, 160), 2)
        else:
            best_iou = 0
            best_box = None
            for box in detections_global:
                x1, y1, x2, y2, label = box
                score = compute_iou(selected_box, (x1, y1, x2, y2))
                if score > best_iou:
                    best_iou = score
                    best_box = (x1, y1, x2, y2)

            if best_iou > 0.1 and best_box:
                bx1, by1, bx2, by2 = best_box
                tracker = cv2.TrackerCSRT_create()
                tracker.init(frame, (bx1, by1, bx2 - bx1, by2 - by1))

    cv2.imshow("Object Detection", frame)   
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('r'):
        tracking = False
        tracker = None
        selected_box = None

cap.release()
cv2.destroyAllWindows()

/Users/rishi/DELTA/Hakc/hack/lib/python3.11/site-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Now tracking: ???


-1

In [ ]:
import cv2
from ultralytics import YOLO

# Initialize YOLOv8
model = YOLO('yolov8n.pt')

tracking = False
tracked_id = None
tracked_label = ""
current_results = None  # Global variable to hold the latest YOLO frame data

def mouse_click(event, x, y, flags, param):
    global tracking, tracked_id, tracked_label, current_results

    if event == cv2.EVENT_LBUTTONDOWN and current_results:
        boxes = current_results[0].boxes
        # Ensure YOLO has actually assigned tracking IDs to the objects in this frame
        if boxes.id is not None:
            for i in range(len(boxes)):
                x1, y1, x2, y2 = map(int, boxes.xyxy[i])
                
                # Check if the click coordinates fall inside this bounding box
                if x1 <= x <= x2 and y1 <= y <= y2:
                    tracked_id = int(boxes.id[i])
                    tracked_label = model.names[int(boxes.cls[i])]
                    tracking = True
                    print(f"Now tracking: {tracked_label} (ID: {tracked_id})")
                    break

cap = cv2.VideoCapture(0)
cv2.namedWindow("Object Detection")
cv2.setMouseCallback("Object Detection", mouse_click)

while True:
    ret, frame = cap.read()
    if not ret:                     
        break
    results = model.track(frame, persist=True, verbose=False)
    current_results = results 

    boxes = results[0].boxes

    if tracking and boxes.id is not None:
        # Look for our saved tracked_id in the current frame
        object_found_this_frame = False
        
        for i in range(len(boxes)):
            current_id = int(boxes.id[i])
            if current_id == tracked_id:
                object_found_this_frame = True
                x1, y1, x2, y2 = map(int, boxes.xyxy[i])
                
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)
                cv2.putText(frame, tracked_label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                break
        
    else:
        # Not tracking anything, so draw all YOLO detections to let the user choose
        if boxes.id is not None:
            for i in range(len(boxes)):
                x1, y1, x2, y2 = map(int, boxes.xyxy[i])
                label = model.names[int(boxes.cls[i])]
                
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    cv2.imshow("Object Detection", frame)   
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('r'):
        # Press 'r' to reset tracking and pick a new target
        tracking = False
        tracked_id = None
        tracked_label = ""

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.5 MB ? eta -:--:--
   ━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.3/1.5 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━ 1.0/1.5 MB 4.1 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━ 1.0/1.5 MB 4.1 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 2.2 MB/s  0:00:00

[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: pip install --upgrade pip

requirements: AutoUpdate success ✅ 2.9s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect

Now tracking: bottle (ID: 136)
Now tracking: person (ID: 153)
Now tracking: person (ID: 177)
Now tracking: person (ID: 206)
Now tracking: person (ID: 221)
